# Advanced Differential Equations in Deep Learning for Robot Control

### Author: Dr. D Bhanu Prakash

This comprehensive implementation covers:
1. Neural ODEs with advanced solvers (RK45, Dopri5)
2. Stochastic Differential Equations (SDEs) with noise modeling
3. Robot control applications (pendulum, manipulator)
4. Model Predictive Control with Neural DEs
5. Detailed mathematical explanations

In [ ]:
"""
Mathematical Foundation:
-----------------------

1. NEURAL ORDINARY DIFFERENTIAL EQUATIONS (Neural ODE):
   
   Standard ODE: dz/dt = f(z(t), t, θ)
   
   Where:
   - z(t) ∈ ℝⁿ is the state at time t
   - f is a neural network with parameters θ
   - We solve: z(t₁) = z(t₀) + ∫[t₀,t₁] f(z(t), t, θ) dt
   
   Key insight: Instead of discrete layers, we have continuous transformations
   
   Adjoint Method for Training:
   - Forward: Solve ODE to get z(t₁)
   - Backward: Solve augmented ODE backwards in time
   - Adjoint state: a(t) = ∂L/∂z(t)
   - da/dt = -a(t)ᵀ ∂f/∂z
   - This saves memory - O(1) instead of O(depth)

2. STOCHASTIC DIFFERENTIAL EQUATIONS (SDE):
   
   General SDE: dz = f(z,t)dt + g(z,t)dW
   
   Where:
   - f(z,t) is the drift term (deterministic)
   - g(z,t) is the diffusion term (stochastic)
   - dW is Wiener process (Brownian motion): dW ~ N(0, dt)
   
   Euler-Maruyama discretization:
   z(t+Δt) = z(t) + f(z,t)Δt + g(z,t)√(Δt)ε, where ε ~ N(0,I)
   
   Why SDEs?
   - Model uncertainty in dynamics
   - Robust to perturbations
   - Better generalization

3. ROBOT DYNAMICS:
   
   General form: M(q)q̈ + C(q,q̇)q̇ + G(q) = τ
   
   Where:
   - q: joint angles (configuration)
   - q̇: joint velocities
   - q̈: joint accelerations
   - M(q): inertia matrix
   - C(q,q̇): Coriolis and centrifugal terms
   - G(q): gravity term
   - τ: control torques
   
   State-space form:
   x = [q, q̇]ᵀ
   dx/dt = [q̇, M⁻¹(τ - C(q,q̇)q̇ - G(q))]ᵀ

4. MODEL PREDICTIVE CONTROL (MPC) with Neural ODE:
   
   Optimization problem:
   min ∑ᵢ [‖z(tᵢ) - z_ref(tᵢ)‖² + λ‖u(tᵢ)‖²]
   
   subject to: dz/dt = f_neural(z,u,t)
               u_min ≤ u ≤ u_max
   
   Where:
   - z_ref is reference trajectory
   - u is control input
   - λ is control cost weight

Dependencies: pip install torch numpy matplotlib scipy
"""

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import odeint
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# ============================================================================
# PART 1: ADVANCED ODE SOLVERS
# ============================================================================

class RK45Solver:
    """
    Runge-Kutta 4/5 adaptive step size solver
    
    Mathematical Details:
    --------------------
    Uses Dormand-Prince method with error estimation:
    
    k₁ = f(t, y)
    k₂ = f(t + c₂h, y + h(a₂₁k₁))
    ...
    k₆ = f(t + c₆h, y + h(a₆₁k₁ + ... + a₆₅k₅))
    
    Fifth-order solution:
    y_{n+1} = yₙ + h(b₁k₁ + b₂k₂ + ... + b₆k₆)
    
    Fourth-order solution (for error):
    ŷ_{n+1} = yₙ + h(b̂₁k₁ + b̂₂k₂ + ... + b̂₆k₆)
    
    Error estimate: e = ‖y_{n+1} - ŷ_{n+1}‖
    
    Adaptive step: h_new = 0.9 * h * (tol/e)^(1/5)
    """
    
    def __init__(self, func, rtol=1e-3, atol=1e-6):
        self.func = func
        self.rtol = rtol
        self.atol = atol
        
        # Dormand-Prince coefficients
        self.c = torch.tensor([0., 1./5, 3./10, 4./5, 8./9, 1.])
        self.a = [
            [],
            [1./5],
            [3./40, 9./40],
            [44./45, -56./15, 32./9],
            [19372./6561, -25360./2187, 64448./6561, -212./729],
            [9017./3168, -355./33, 46732./5247, 49./176, -5103./18656]
        ]
        self.b = torch.tensor([35./384, 0., 500./1113, 125./192, -2187./6784, 11./84])
        self.b_hat = torch.tensor([5179./57600, 0., 7571./16695, 393./640, -92097./339200, 187./2100, 1./40])
    
    def step(self, t, y, h):
        """Single RK45 step with error estimation"""
        k = []
        k.append(self.func(t, y))
        
        for i in range(1, 6):
            t_i = t + self.c[i] * h
            y_i = y.clone()
            for j, a_ij in enumerate(self.a[i]):
                y_i = y_i + h * a_ij * k[j]
            k.append(self.func(t_i, y_i))
        
        # Fifth-order solution
        y_new = y.clone()
        for i, b_i in enumerate(self.b):
            y_new = y_new + h * b_i * k[i]
        
        # Fourth-order solution for error estimate
        y_hat = y.clone()
        for i, b_hat_i in enumerate(self.b_hat):
            y_hat = y_hat + h * b_hat_i * k[i]
        
        # Error estimate
        error = torch.abs(y_new - y_hat).max()
        
        return y_new, error
    
    def integrate(self, y0, t_span, max_steps=1000):
        """Adaptive integration from t_span[0] to t_span[1]"""
        t = t_span[0]
        y = y0
        h = 0.1  # initial step size
        
        trajectory = [y0.clone()]
        times = [t]
        
        steps = 0
        while t < t_span[1] and steps < max_steps:
            # Adjust final step
            if t + h > t_span[1]:
                h = t_span[1] - t
            
            # Take step
            y_new, error = self.step(t, y, h)
            
            # Compute tolerance
            tol = self.atol + self.rtol * torch.abs(y).max()
            
            # Accept or reject step
            if error < tol or h < 1e-8:
                t += h
                y = y_new
                trajectory.append(y.clone())
                times.append(t)
                steps += 1
            
            # Adapt step size
            if error > 0:
                h = 0.9 * h * (tol / error) ** 0.2
                h = min(h, 0.5)  # max step
            
        return torch.stack(trajectory), torch.tensor(times)

In [ ]:
# ============================================================================
# PART 2: STOCHASTIC DIFFERENTIAL EQUATIONS (SDE)
# ============================================================================

class NeuralSDE(nn.Module):
    """
    Neural Stochastic Differential Equation
    
    Mathematical Form:
    -----------------
    dz = f_drift(z, t, θ) dt + g_diffusion(z, t, θ) dW
    
    Where:
    - f_drift: deterministic dynamics (neural network)
    - g_diffusion: stochastic dynamics (neural network)
    - dW: Wiener process increment
    
    Discretization (Euler-Maruyama):
    z(t+Δt) = z(t) + f_drift(z,t)Δt + g_diffusion(z,t)√Δt·ε
    where ε ~ N(0, I)
    
    Applications:
    - Modeling uncertainty in robot dynamics
    - Robust control under perturbations
    - Learning stochastic processes
    """
    
    def __init__(self, state_dim, hidden_dim=64, noise_type='diagonal'):
        super().__init__()
        self.state_dim = state_dim
        self.noise_type = noise_type
        
        # Drift network: f_drift(z, t)
        self.drift_net = nn.Sequential(
            nn.Linear(state_dim + 1, hidden_dim),  # +1 for time
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, state_dim)
        )
        
        # Diffusion network: g_diffusion(z, t)
        if noise_type == 'diagonal':
            # Output diagonal elements: g(z,t) = diag(σ₁, σ₂, ..., σₙ)
            self.diffusion_net = nn.Sequential(
                nn.Linear(state_dim + 1, hidden_dim),
                nn.Tanh(),
                nn.Linear(hidden_dim, state_dim),
                nn.Softplus()  # ensure positive
            )
        elif noise_type == 'general':
            # Output full matrix: g(z,t) ∈ ℝⁿˣⁿ
            self.diffusion_net = nn.Sequential(
                nn.Linear(state_dim + 1, hidden_dim),
                nn.Tanh(),
                nn.Linear(hidden_dim, state_dim * state_dim)
            )
    
    def drift(self, z, t):
        """Compute drift term f(z, t)"""
        t_expand = t * torch.ones(z.shape[0], 1)
        inp = torch.cat([z, t_expand], dim=1)
        return self.drift_net(inp)
    
    def diffusion(self, z, t):
        """Compute diffusion term g(z, t)"""
        t_expand = t * torch.ones(z.shape[0], 1)
        inp = torch.cat([z, t_expand], dim=1)
        
        if self.noise_type == 'diagonal':
            # Return diagonal matrix as vector
            return self.diffusion_net(inp)
        else:
            # Return full matrix
            g_flat = self.diffusion_net(inp)
            return g_flat.reshape(-1, self.state_dim, self.state_dim)
    
    def forward(self, z0, t_span, dt=0.01, return_path=False):
        """
        Integrate SDE using Euler-Maruyama method
        
        z(t+dt) = z(t) + f(z,t)dt + g(z,t)√dt·ε
        """
        t = t_span[0]
        z = z0
        
        if return_path:
            trajectory = [z0.clone()]
            times = [t]
        
        while t < t_span[1]:
            dt_step = min(dt, t_span[1] - t)
            
            # Drift term
            drift = self.drift(z, t)
            
            # Diffusion term
            diffusion = self.diffusion(z, t)
            
            # Wiener process increment: dW ~ N(0, dt)
            dW = torch.randn_like(z) * torch.sqrt(torch.tensor(dt_step))
            
            # Euler-Maruyama step
            if self.noise_type == 'diagonal':
                z = z + drift * dt_step + diffusion * dW
            else:
                # Matrix multiplication for general noise
                z = z + drift * dt_step + torch.bmm(diffusion, dW.unsqueeze(-1)).squeeze(-1)
            
            t += dt_step
            
            if return_path:
                trajectory.append(z.clone())
                times.append(t)
        
        if return_path:
            return torch.stack(trajectory), torch.tensor(times)
        return z

In [ ]:
# ============================================================================
# PART 3: ROBOT CONTROL - INVERTED PENDULUM
# ============================================================================

class PendulumDynamics:
    """
    Inverted Pendulum Dynamics
    
    Mathematical Model:
    ------------------
    State: x = [θ, θ̇]ᵀ
    - θ: angle from vertical (rad)
    - θ̇: angular velocity (rad/s)
    
    Equation of motion:
    θ̈ = (g/L)sin(θ) - (b/mL²)θ̇ + (1/mL²)τ
    
    Where:
    - m: mass (kg)
    - L: length (m)
    - g: gravity (9.81 m/s²)
    - b: damping coefficient
    - τ: control torque (N·m)
    
    State-space form:
    dx/dt = [θ̇, (g/L)sin(θ) - (b/mL²)θ̇ + (1/mL²)τ]ᵀ
    """
    
    def __init__(self, mass=1.0, length=1.0, damping=0.1):
        self.m = mass
        self.L = length
        self.b = damping
        self.g = 9.81
    
    def dynamics(self, state, control):
        """
        Compute dx/dt given state and control
        
        Args:
            state: [batch, 2] - [θ, θ̇]
            control: [batch, 1] - [τ]
        
        Returns:
            [batch, 2] - [dθ/dt, dθ̇/dt]
        """
        theta = state[:, 0:1]
        theta_dot = state[:, 1:2]
        
        # θ̈ = (g/L)sin(θ) - (b/mL²)θ̇ + (1/mL²)τ
        theta_ddot = (self.g / self.L) * torch.sin(theta) - \
                     (self.b / (self.m * self.L**2)) * theta_dot + \
                     (1.0 / (self.m * self.L**2)) * control
        
        return torch.cat([theta_dot, theta_ddot], dim=1)


class NeuralControllerODE(nn.Module):
    """
    Neural ODE-based controller for inverted pendulum
    
    Architecture:
    ------------
    1. State encoder: maps observations to latent space
    2. Neural ODE: evolves latent state forward in time
    3. Policy network: maps latent state to control action
    
    Training:
    --------
    - Minimize tracking error: L = Σ ‖θ(t) - θ_ref(t)‖²
    - Control cost: + λ Σ ‖τ(t)‖²
    - Energy efficiency: + γ Σ |θ̇(t)|
    """
    
    def __init__(self, state_dim=2, latent_dim=8, hidden_dim=64):
        super().__init__()
        
        self.state_dim = state_dim
        self.latent_dim = latent_dim
        
        # State encoder
        self.encoder = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        
        # Neural ODE function
        self.ode_func = nn.Sequential(
            nn.Linear(latent_dim + 1, hidden_dim),  # +1 for time
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, latent_dim)
        )
        
        # Policy network (outputs control)
        self.policy = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Tanh()  # bounded control
        )
        
        self.max_torque = 2.0
    
    def ode_forward(self, t, z):
        """ODE function: dz/dt = f(z, t)"""
        batch_size = z.shape[0]
        t_vec = t * torch.ones(batch_size, 1)
        inp = torch.cat([z, t_vec], dim=1)
        return self.ode_func(inp)
    
    def get_control(self, state):
        """Compute control action from state"""
        z = self.encoder(state)
        u = self.policy(z) * self.max_torque
        return u
    
    def forward(self, state0, t_span, dt=0.05):
        """
        Roll out trajectory with neural controller
        
        Process:
        1. Encode initial state: z₀ = encoder(x₀)
        2. For each time step:
           a. Compute control: u = policy(z)
           b. Apply control to real dynamics
           c. Evolve latent state with ODE
        """
        # Initialize
        z = self.encoder(state0)
        state = state0.clone()
        
        t = t_span[0]
        trajectory = [state0.clone()]
        controls = []
        
        # Dynamics model
        dynamics = PendulumDynamics()
        
        while t < t_span[1]:
            # Get control action
            u = self.policy(z) * self.max_torque
            controls.append(u.clone())
            
            # Update real state using true dynamics
            dx = dynamics.dynamics(state, u)
            state = state + dx * dt
            
            # Update latent state using Neural ODE (Euler step)
            dz = self.ode_forward(t, z)
            z = z + dz * dt
            
            t += dt
            trajectory.append(state.clone())
        
        return torch.stack(trajectory), torch.stack(controls)


def train_pendulum_controller():
    """
    Train Neural ODE controller for inverted pendulum
    
    Goal: Balance pendulum at upright position (θ = 0)
    """
    print("=" * 80)
    print("ROBOT CONTROL: Neural ODE Controller for Inverted Pendulum")
    print("=" * 80)
    print()
    print("Mathematical Setup:")
    print("-" * 80)
    print("Dynamics: θ̈ = (g/L)sin(θ) - (b/mL²)θ̇ + (1/mL²)τ")
    print("State: x = [θ, θ̇]ᵀ")
    print("Control: τ ∈ [-2, 2] N·m")
    print("Objective: Minimize ‖θ - θ_ref‖² + λ‖τ‖²")
    print("-" * 80)
    print()
    
    # Create model
    controller = NeuralControllerODE(state_dim=2, latent_dim=8, hidden_dim=64)
    optimizer = optim.Adam(controller.parameters(), lr=0.001)
    
    # Training parameters
    n_episodes = 500
    batch_size = 32
    t_horizon = 5.0
    dt = 0.05
    
    losses = []
    
    for episode in range(n_episodes):
        optimizer.zero_grad()
        
        # Sample initial conditions (pendulum starts at various angles)
        theta0 = torch.rand(batch_size, 1) * 2 * np.pi - np.pi  # [-π, π]
        theta_dot0 = torch.rand(batch_size, 1) * 2 - 1  # [-1, 1]
        state0 = torch.cat([theta0, theta_dot0], dim=1)
        
        # Roll out trajectory
        trajectory, controls = controller(state0, [0.0, t_horizon], dt=dt)
        
        # Reference: upright position
        theta_ref = torch.zeros_like(trajectory[..., 0:1])
        
        # Compute loss
        # 1. Tracking error
        tracking_loss = torch.mean((trajectory[..., 0:1] - theta_ref) ** 2)
        
        # 2. Control cost
        control_loss = 0.1 * torch.mean(controls ** 2)
        
        # 3. Velocity penalty (smooth motion)
        velocity_loss = 0.01 * torch.mean(trajectory[..., 1:2] ** 2)
        
        # Total loss
        loss = tracking_loss + control_loss + velocity_loss
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(controller.parameters(), 1.0)
        optimizer.step()
        
        losses.append(loss.item())
        
        if (episode + 1) % 100 == 0:
            print(f"Episode {episode+1}/{n_episodes}")
            print(f"  Total Loss: {loss.item():.6f}")
            print(f"  Tracking: {tracking_loss.item():.6f}")
            print(f"  Control: {control_loss.item():.6f}")
            print(f"  Velocity: {velocity_loss.item():.6f}")
            print()
    
    # Test controller
    print("Testing controller on new initial conditions...")
    controller.eval()
    
    with torch.no_grad():
        # Test from challenging initial condition
        state0_test = torch.tensor([[np.pi * 0.8, 0.0]])  # Start at ~144 degrees
        trajectory_test, controls_test = controller(state0_test, [0.0, 10.0], dt=0.02)
        
        traj_np = trajectory_test.squeeze().numpy()
        ctrl_np = controls_test.squeeze().numpy()
        times = np.arange(len(traj_np)) * 0.02
    
    # Visualization
    fig = plt.figure(figsize=(15, 10))
    
    # Plot 1: Training loss
    ax1 = plt.subplot(3, 2, 1)
    ax1.plot(losses, linewidth=2)
    ax1.set_xlabel('Episode', fontsize=12)
    ax1.set_ylabel('Total Loss', fontsize=12)
    ax1.set_title('Training Progress', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.set_yscale('log')
    
    # Plot 2: Angle trajectory
    ax2 = plt.subplot(3, 2, 2)
    ax2.plot(times, np.rad2deg(traj_np[:, 0]), linewidth=2, label='θ (angle)')
    ax2.axhline(y=0, color='r', linestyle='--', label='Target (0°)', linewidth=2)
    ax2.set_xlabel('Time (s)', fontsize=12)
    ax2.set_ylabel('Angle (degrees)', fontsize=12)
    ax2.set_title('Pendulum Angle vs Time', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Angular velocity
    ax3 = plt.subplot(3, 2, 3)
    ax3.plot(times, traj_np[:, 1], linewidth=2, color='orange')
    ax3.set_xlabel('Time (s)', fontsize=12)
    ax3.set_ylabel('Angular Velocity (rad/s)', fontsize=12)
    ax3.set_title('Angular Velocity vs Time', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Control input
    ax4 = plt.subplot(3, 2, 4)
    ctrl_times = np.arange(len(ctrl_np)) * 0.02
    ax4.plot(ctrl_times, ctrl_np, linewidth=2, color='green')
    ax4.set_xlabel('Time (s)', fontsize=12)
    ax4.set_ylabel('Torque (N·m)', fontsize=12)
    ax4.set_title('Control Input vs Time', fontsize=14, fontweight='bold')
    ax4.grid(True, alpha=0.3)
    ax4.axhline(y=2.0, color='r', linestyle='--', alpha=0.5, label='Max torque')
    ax4.axhline(y=-2.0, color='r', linestyle='--', alpha=0.5)
    ax4.legend(fontsize=10)
    
    # Plot 5: Phase portrait
    ax5 = plt.subplot(3, 2, 5)
    ax5.plot(np.rad2deg(traj_np[:, 0]), traj_np[:, 1], linewidth=2)
    ax5.scatter(np.rad2deg(traj_np[0, 0]), traj_np[0, 1], 
                c='green', s=100, marker='o', label='Start', zorder=5)
    ax5.scatter(0, 0, c='red', s=100, marker='*', label='Target', zorder=5)
    ax5.set_xlabel('Angle (degrees)', fontsize=12)
    ax5.set_ylabel('Angular Velocity (rad/s)', fontsize=12)
    ax5.set_title('Phase Portrait', fontsize=14, fontweight='bold')
    ax5.legend(fontsize=10)
    ax5.grid(True, alpha=0.3)
    
    # Plot 6: Energy analysis
    ax6 = plt.subplot(3, 2, 6)
    m, L, g = 1.0, 1.0, 9.81
    potential_energy = m * g * L * (1 - np.cos(traj_np[:, 0]))  # PE = mgL(1-cosθ)
    kinetic_energy = 0.5 * m * L**2 * traj_np[:, 1]**2  # KE = ½mL²θ̇²
    total_energy = potential_energy + kinetic_energy
    
    ax6.plot(times, potential_energy, linewidth=2, label='Potential Energy', alpha=0.7)
    ax6.plot(times, kinetic_energy, linewidth=2, label='Kinetic Energy', alpha=0.7)
    ax6.plot(times, total_energy, linewidth=2, label='Total Energy', linestyle='--')
    ax6.set_xlabel('Time (s)', fontsize=12)
    ax6.set_ylabel('Energy (J)', fontsize=12)
    ax6.set_title('Energy Analysis', fontsize=14, fontweight='bold')
    ax6.legend(fontsize=10)
    ax6.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('pendulum_control_results.png', dpi=150, bbox_inches='tight')
    print("\nResults saved to 'pendulum_control_results.png'")
    plt.close()
    
    return controller

In [ ]:
# ============================================================================
# PART 4: STOCHASTIC ROBOT CONTROL with Uncertainty
# ============================================================================

def train_stochastic_pendulum():
    """
    Train Neural SDE for robust pendulum control under uncertainty
    
    Mathematical Framework:
    ----------------------
    State dynamics with noise:
    dx = f(x, u, t)dt + g(x, t)dW
    
    where g(x, t) models:
    - Model uncertainty
    - External disturbances
    - Sensor noise
    
    Training:
    --------
    Learn both drift and diffusion simultaneously
    Loss = E[‖x - x_ref‖²] + λE[‖u‖²] + β·KL[q(noise)||p(noise)]
    """
    print("\n" + "=" * 80)
    print("STOCHASTIC ROBOT CONTROL: Neural SDE for Robust Pendulum Control")
    print("=" * 80)
    print()
    print("Mathematical Setup:")
    print("-" * 80)
    print("Stochastic Dynamics: dx = f(x,u)dt + g(x)dW")
    print("where:")
    print("  f(x,u) = drift (controlled dynamics)")
    print("  g(x) = diffusion (uncertainty/noise)")
    print("  dW = Wiener process (Brownian motion)")
    print()
    print("Discretization (Euler-Maruyama):")
    print("  x(t+Δt) = x(t) + f(x,u)Δt + g(x)√Δt·ε, ε ~ N(0,I)")
    print("-" * 80)
    print()
    
    # Create stochastic model
    sde_model = NeuralSDE(state_dim=2, hidden_dim=32, noise_type='diagonal')
    
    # Policy network (separate from SDE)
    policy = nn.Sequential(
        nn.Linear(2, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Tanh()
    )
    
    optimizer = optim.Adam(list(sde_model.parameters()) + list(policy.parameters()), 
                          lr=0.002)
    
    # Training parameters
    n_episodes = 300
    batch_size = 16
    t_horizon = 5.0
    dt = 0.05
    max_torque = 2.0
    
    losses = []
    noise_scales = []
    
    dynamics = PendulumDynamics()
    
    for episode in range(n_episodes):
        optimizer.zero_grad()
        
        # Sample initial conditions
        theta0 = torch.rand(batch_size, 1) * np.pi - np.pi/2  # [-π/2, π/2]
        theta_dot0 = torch.rand(batch_size, 1) * 0.5 - 0.25
        state = torch.cat([theta0, theta_dot0], dim=1)
        
        # Roll out trajectory with stochastic dynamics
        trajectory = [state.clone()]
        controls = []
        
        t = 0.0
        while t < t_horizon:
            # Compute control
            u = policy(state) * max_torque
            controls.append(u)
            
            # Deterministic dynamics
            f_det = dynamics.dynamics(state, u)
            
            # Stochastic component from Neural SDE
            g_stoch = sde_model.diffusion(state, t)
            dW = torch.randn_like(state) * np.sqrt(dt)
            
            # Combined update (Euler-Maruyama)
            state = state + f_det * dt + g_stoch * dW
            
            trajectory.append(state.clone())
            t += dt
        
        trajectory = torch.stack(trajectory)
        controls = torch.stack(controls)
        
        # Losses
        theta_ref = torch.zeros_like(trajectory[..., 0:1])
        tracking_loss = torch.mean((trajectory[..., 0:1] - theta_ref) ** 2)
        control_loss = 0.05 * torch.mean(controls ** 2)
        
        # Penalize excessive noise (we want minimal necessary uncertainty)
        noise_penalty = 0.01 * torch.mean(g_stoch ** 2)
        
        loss = tracking_loss + control_loss + noise_penalty
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(list(sde_model.parameters()) + 
                                      list(policy.parameters()), 1.0)
        optimizer.step()
        
        losses.append(loss.item())
        noise_scales.append(torch.mean(g_stoch).item())
        
        if (episode + 1) % 60 == 0:
            print(f"Episode {episode+1}/{n_episodes}")
            print(f"  Loss: {loss.item():.6f}")
            print(f"  Tracking: {tracking_loss.item():.6f}")
            print(f"  Avg Noise Scale: {noise_scales[-1]:.6f}")
            print()
    
    # Test with multiple rollouts to show uncertainty
    print("Testing stochastic controller with 20 rollouts...")
    sde_model.eval()
    policy.eval()
    
    n_rollouts = 20
    all_trajectories = []
    
    with torch.no_grad():
        state0_test = torch.tensor([[np.pi * 0.5, 0.0]])  # Start at 90 degrees
        
        for _ in range(n_rollouts):
            state = state0_test.clone()
            traj = [state.clone()]
            
            t = 0.0
            while t < 8.0:
                u = policy(state) * max_torque
                f_det = dynamics.dynamics(state, u)
                g_stoch = sde_model.diffusion(state, t)
                dW = torch.randn_like(state) * np.sqrt(0.05)
                
                state = state + f_det * 0.05 + g_stoch * dW
                traj.append(state.clone())
                t += 0.05
            
            all_trajectories.append(torch.stack(traj).squeeze().numpy())
    
    # Visualization
    fig = plt.figure(figsize=(15, 10))
    
    # Plot 1: Training loss
    ax1 = plt.subplot(2, 3, 1)
    ax1.plot(losses, linewidth=2, color='blue')
    ax1.set_xlabel('Episode', fontsize=12)
    ax1.set_ylabel('Total Loss', fontsize=12)
    ax1.set_title('Training Progress (SDE)', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Learned noise scale
    ax2 = plt.subplot(2, 3, 2)
    ax2.plot(noise_scales, linewidth=2, color='red')
    ax2.set_xlabel('Episode', fontsize=12)
    ax2.set_ylabel('Avg Diffusion Scale', fontsize=12)
    ax2.set_title('Learned Noise Intensity', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Multiple stochastic rollouts (angle)
    ax3 = plt.subplot(2, 3, 3)
    times = np.arange(all_trajectories[0].shape[0]) * 0.05
    for traj in all_trajectories:
        ax3.plot(times, np.rad2deg(traj[:, 0]), alpha=0.4, linewidth=1)
    
    # Mean trajectory
    mean_traj = np.mean([traj[:, 0] for traj in all_trajectories], axis=0)
    std_traj = np.std([traj[:, 0] for traj in all_trajectories], axis=0)
    ax3.plot(times, np.rad2deg(mean_traj), 'b-', linewidth=3, label='Mean')
    ax3.fill_between(times, 
                     np.rad2deg(mean_traj - std_traj), 
                     np.rad2deg(mean_traj + std_traj),
                     alpha=0.3, label='±1 std')
    ax3.axhline(y=0, color='r', linestyle='--', linewidth=2, label='Target')
    ax3.set_xlabel('Time (s)', fontsize=12)
    ax3.set_ylabel('Angle (degrees)', fontsize=12)
    ax3.set_title('Stochastic Trajectories (20 rollouts)', fontsize=14, fontweight='bold')
    ax3.legend(fontsize=9)
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Phase portrait with uncertainty
    ax4 = plt.subplot(2, 3, 4)
    for traj in all_trajectories:
        ax4.plot(np.rad2deg(traj[:, 0]), traj[:, 1], alpha=0.3, linewidth=1)
    ax4.scatter(0, 0, c='red', s=200, marker='*', label='Target', zorder=5)
    ax4.scatter(np.rad2deg(all_trajectories[0][0, 0]), 
               all_trajectories[0][0, 1],
               c='green', s=100, marker='o', label='Start', zorder=5)
    ax4.set_xlabel('Angle (degrees)', fontsize=12)
    ax4.set_ylabel('Angular Velocity (rad/s)', fontsize=12)
    ax4.set_title('Phase Portrait (with Noise)', fontsize=14, fontweight='bold')
    ax4.legend(fontsize=10)
    ax4.grid(True, alpha=0.3)
    
    # Plot 5: Uncertainty quantification
    ax5 = plt.subplot(2, 3, 5)
    angle_std = np.std([traj[:, 0] for traj in all_trajectories], axis=0)
    ax5.plot(times, np.rad2deg(angle_std), linewidth=2, color='purple')
    ax5.set_xlabel('Time (s)', fontsize=12)
    ax5.set_ylabel('Std Dev (degrees)', fontsize=12)
    ax5.set_title('Uncertainty Evolution', fontsize=14, fontweight='bold')
    ax5.grid(True, alpha=0.3)
    ax5.fill_between(times, 0, np.rad2deg(angle_std), alpha=0.3)
    
    # Plot 6: Comparison with/without noise at key timesteps
    ax6 = plt.subplot(2, 3, 6)
    timesteps = [0, 50, 100, 150]  # indices
    angles_at_times = [[traj[t, 0] for traj in all_trajectories] for t in timesteps]
    
    positions = np.arange(len(timesteps))
    bp = ax6.boxplot(angles_at_times, positions=positions, widths=0.6,
                     patch_artist=True, showfliers=False)
    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')
    
    ax6.set_xticks(positions)
    ax6.set_xticklabels([f't={times[t]:.1f}s' for t in timesteps])
    ax6.set_ylabel('Angle (rad)', fontsize=12)
    ax6.set_title('Uncertainty Distribution Over Time', fontsize=14, fontweight='bold')
    ax6.axhline(y=0, color='r', linestyle='--', linewidth=2, alpha=0.7)
    ax6.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('stochastic_pendulum_results.png', dpi=150, bbox_inches='tight')
    print("Results saved to 'stochastic_pendulum_results.png'")
    plt.close()

In [ ]:
# ============================================================================
# PART 5: MODEL PREDICTIVE CONTROL with Neural ODE
# ============================================================================

class NeuralMPC:
    """
    Model Predictive Control using Neural ODE
    
    Mathematical Formulation:
    ------------------------
    At each time step t, solve:
    
    min  Σᵢ [‖z(tᵢ) - z_ref(tᵢ)‖²_Q + ‖u(tᵢ)‖²_R]
    u
    
    subject to:
        dz/dt = f_neural(z, u, t)  (Neural ODE dynamics)
        u_min ≤ u ≤ u_max           (Control constraints)
        z(t₀) = z_current           (Initial condition)
    
    Where:
    - z: state trajectory
    - u: control sequence
    - Q: state cost matrix
    - R: control cost matrix
    - Horizon: T = N·Δt
    
    Implementation:
    --------------
    1. Learn dynamics model: Neural ODE
    2. At each timestep:
       a. Use current state as initial condition
       b. Optimize control sequence u[0:N] via gradient descent
       c. Apply first control u[0]
       d. Repeat (receding horizon)
    """
    
    def __init__(self, state_dim=2, control_dim=1, horizon=10, dt=0.1):
        self.state_dim = state_dim
        self.control_dim = control_dim
        self.horizon = horizon
        self.dt = dt
        
        # Neural ODE dynamics model
        self.dynamics_model = nn.Sequential(
            nn.Linear(state_dim + control_dim + 1, 64),  # +1 for time
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, state_dim)
        )
        
        # Cost matrices
        self.Q = torch.eye(state_dim, dtype=torch.float32) * 10.0  # state cost
        self.R = torch.eye(control_dim, dtype=torch.float32) * 0.1  # control cost
        
        self.max_control = 2.0

    def dynamics(self, state, control, t):
        """Learned dynamics: dx/dt = f(x, u, t)"""
        batch_size = state.shape[0]
        t_tensor = torch.tensor([[t]], dtype=torch.float32).expand(batch_size, 1)  # Fix: expand to match batch size
        inp = torch.cat([state, control, t_tensor], dim=1)
        return self.dynamics_model(inp)
    
    def rollout(self, state0, control_sequence, t0):
        """
        Roll out trajectory given control sequence
        
        Args:
            state0: [1, state_dim] initial state
            control_sequence: [horizon, 1, control_dim] controls
            t0: initial time
        
        Returns:
            trajectory: [horizon+1, 1, state_dim]
        """
        state = state0
        trajectory = [state0]
        
        for i, u in enumerate(control_sequence):
            t = t0 + i * self.dt
            dx = self.dynamics(state, u, t)
            state = state + dx * self.dt
            trajectory.append(state)
        
        return torch.stack(trajectory)

    def compute_cost(self, trajectory, controls, reference):
        """
        Compute MPC cost function
    
        J = Σ (z - z_ref)ᵀQ(z - z_ref) + uᵀRu
        Args:
            trajectory: [horizon+1, 1, state_dim]
            controls: [horizon, 1, control_dim]
            reference: [horizon+1, 1, state_dim]
        """
        # Ensure consistent dtype
        trajectory = trajectory.float()
        reference = reference.float()
        controls = controls.float()
    
        # State cost
        state_error = trajectory - reference
        state_cost = 0.0
        for i in range(len(trajectory)):
            error = state_error[i].view(-1)  # Flatten to 1D vector
            # Quadratic form: error^T @ Q @ error
            state_cost += (error @ self.Q @ error)
    
        # Control cost
        control_cost = 0.0
        for u in controls:
            u_vec = u.view(-1)  # Flatten to 1D vector
            # Quadratic form: u^T @ R @ u
            control_cost += (u_vec @ self.R @ u_vec)
    
        return state_cost + control_cost
    
    def optimize_controls(self, state0, reference, t0, n_iterations=20):
        """
        Optimize control sequence for MPC
        
        Uses gradient descent to minimize cost function
        """
        # Initialize control sequence (learnable parameters)
        controls = torch.zeros(self.horizon, 1, self.control_dim, requires_grad=True)
        
        optimizer = optim.Adam([controls], lr=0.1)
        
        for iteration in range(n_iterations):
            optimizer.zero_grad()
            
            # Apply control constraints via tanh
            controls_bounded = torch.tanh(controls) * self.max_control
            
            # Roll out trajectory
            trajectory = self.rollout(state0, controls_bounded, t0)
            
            # Compute cost
            cost = self.compute_cost(trajectory, controls_bounded, reference)
            
            # Optimize
            cost.backward()
            optimizer.step()
        
        # Return optimized controls
        with torch.no_grad():
            optimal_controls = torch.tanh(controls) * self.max_control
        
        return optimal_controls


def train_and_test_mpc():
    """
    Train Neural MPC for pendulum control
    
    Process:
    1. Learn dynamics model from data
    2. Use learned model for MPC optimization
    3. Test on tracking task
    """
    print("\n" + "=" * 80)
    print("MODEL PREDICTIVE CONTROL: Neural ODE-based MPC")
    print("=" * 80)
    print()
    print("Mathematical Framework:")
    print("-" * 80)
    print("Optimization Problem (solved at each timestep):")
    print()
    print("  min  Σᵢ₌₀ᴺ [‖xᵢ - x_ref,ᵢ‖²_Q + ‖uᵢ‖²_R]")
    print("  u")
    print()
    print("  subject to:")
    print("    dx/dt = f_neural(x, u, t)")
    print("    u_min ≤ u ≤ u_max")
    print("    x₀ = x_current")
    print()
    print("Receding Horizon: Apply u₀, observe new state, re-optimize")
    print("-" * 80)
    print()
    
    # Step 1: Generate training data for dynamics model
    print("Step 1: Collecting data for dynamics model...")
    dynamics = PendulumDynamics()
    
    n_samples = 1000
    states = []
    controls = []
    next_states = []
    
    for _ in range(n_samples):
        # Random state
        theta = np.random.uniform(-np.pi, np.pi)
        theta_dot = np.random.uniform(-2, 2)
        state = torch.tensor([[theta, theta_dot]], dtype=torch.float32)
        
        # Random control
        u = torch.tensor([[np.random.uniform(-2, 2)]], dtype=torch.float32)
        
        # Next state
        dx = dynamics.dynamics(state, u)
        next_state = state + dx * 0.1
        
        states.append(state)
        controls.append(u)
        next_states.append(next_state)
    
    states = torch.cat(states)
    controls = torch.cat(controls)
    next_states = torch.cat(next_states)
    
    # Step 2: Train dynamics model
    print("Step 2: Training dynamics model...")
    mpc = NeuralMPC(state_dim=2, control_dim=1, horizon=15, dt=0.1)
    optimizer = optim.Adam(mpc.dynamics_model.parameters(), lr=0.001)
    
    for epoch in range(500):
        optimizer.zero_grad()
        
        # Predict next states
        pred_dx = mpc.dynamics(states, controls, 0.0)
        pred_next_states = states + pred_dx * 0.1
        
        # Loss
        loss = torch.mean((pred_next_states - next_states) ** 2)
        
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 100 == 0:
            print(f"  Epoch {epoch+1}/500, Dynamics Loss: {loss.item():.6f}")
    
    print("\nStep 3: Running MPC controller...")
    
    # Step 3: Test MPC
    # Track a reference trajectory
    t_sim = 10.0
    dt = 0.1
    n_steps = int(t_sim / dt)
    
    # Initial state
    state = torch.tensor([[np.pi * 0.7, 0.0]], dtype=torch.float32)
    
    # Reference trajectory (sinusoidal)
    times = np.linspace(0, t_sim, n_steps)
    ref_angles = 0.3 * np.sin(2 * np.pi * 0.2 * times)  # slow oscillation
    
    trajectory = [state.clone()]
    controls_applied = []
    computation_times = []
    
    import time as time_module
    
    for i in range(n_steps - 1):
        t = i * dt

        # Create reference for horizon
        ref_traj = []
        for j in range(mpc.horizon + 1):
            idx = min(i + j, n_steps - 1)
            ref_state = torch.tensor([[ref_angles[idx], 0.0]], dtype=torch.float32)  # Added dtype
            ref_traj.append(ref_state)
        ref_traj = torch.stack(ref_traj)
        
        # Optimize controls
        start_time = time_module.time()
        optimal_controls = mpc.optimize_controls(state, ref_traj, t, n_iterations=15)
        comp_time = time_module.time() - start_time
        computation_times.append(comp_time)
        
        # Apply first control
        u = optimal_controls[0]
        controls_applied.append(u.clone())
        
        # Update state using TRUE dynamics
        dx = dynamics.dynamics(state, u)
        state = state + dx * dt
        trajectory.append(state.clone())
    

    trajectory = torch.stack(trajectory).squeeze().detach().numpy()
    controls_applied = torch.stack(controls_applied).squeeze().detach().numpy()
    
    # Trim trajectory to match
    trajectory = trajectory[:-1]  # Remove last point to match controls length

    print(f"Average computation time per step: {np.mean(computation_times)*1000:.2f} ms")
    
    # Visualization
    fig = plt.figure(figsize=(15, 10))
    
    # Plot 1: Tracking performance
    ax1 = plt.subplot(2, 3, 1)
    ax1.plot(times[:-1], np.rad2deg(trajectory[:, 0]), linewidth=2, label='Actual')
    ax1.plot(times, np.rad2deg(ref_angles), 'r--', linewidth=2, label='Reference')
    ax1.set_xlabel('Time (s)', fontsize=12)
    ax1.set_ylabel('Angle (degrees)', fontsize=12)
    ax1.set_title('MPC Tracking Performance', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Tracking error
    ax2 = plt.subplot(2, 3, 2)
    tracking_error = np.rad2deg(trajectory[:, 0]) - np.rad2deg(ref_angles[:-1])
    ax2.plot(times[:-1], tracking_error, linewidth=2, color='red')
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Time (s)', fontsize=12)
    ax2.set_ylabel('Error (degrees)', fontsize=12)
    ax2.set_title('Tracking Error', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    rms_error = np.sqrt(np.mean(tracking_error**2))
    ax2.text(0.02, 0.98, f'RMS Error: {rms_error:.2f}°', 
            transform=ax2.transAxes, fontsize=11,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))
    
    # Plot 3: Control input
    ax3 = plt.subplot(2, 3, 3)
    ax3.plot(times[:-1], controls_applied, linewidth=2, color='green')
    ax3.axhline(y=2.0, color='r', linestyle='--', alpha=0.5, label='Limits')
    ax3.axhline(y=-2.0, color='r', linestyle='--', alpha=0.5)
    ax3.set_xlabel('Time (s)', fontsize=12)
    ax3.set_ylabel('Torque (N·m)', fontsize=12)
    ax3.set_title('MPC Control Input', fontsize=14, fontweight='bold')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Phase portrait
    ax4 = plt.subplot(2, 3, 4)
    ax4.plot(np.rad2deg(trajectory[:, 0]), trajectory[:, 1], linewidth=2)
    ax4.scatter(np.rad2deg(trajectory[0, 0]), trajectory[0, 1], 
               c='green', s=100, marker='o', label='Start', zorder=5)
    ax4.plot(np.rad2deg(ref_angles), np.zeros_like(ref_angles), 
            'r--', linewidth=2, alpha=0.7, label='Reference path')
    ax4.set_xlabel('Angle (degrees)', fontsize=12)
    ax4.set_ylabel('Angular Velocity (rad/s)', fontsize=12)
    ax4.set_title('Phase Portrait', fontsize=14, fontweight='bold')
    ax4.legend(fontsize=10)
    ax4.grid(True, alpha=0.3)
    
    # Plot 5: Computation time
    ax5 = plt.subplot(2, 3, 5)
    ax5.plot(times[:-1], np.array(computation_times) * 1000, linewidth=2, color='purple')
    ax5.set_xlabel('Time (s)', fontsize=12)
    ax5.set_ylabel('Computation Time (ms)', fontsize=12)
    ax5.set_title('MPC Computation Time', fontsize=14, fontweight='bold')
    ax5.grid(True, alpha=0.3)
    ax5.axhline(y=np.mean(computation_times)*1000, color='r', 
               linestyle='--', label=f'Mean: {np.mean(computation_times)*1000:.1f} ms')
    ax5.legend(fontsize=10)
    
    # Plot 6: Control effort
    ax6 = plt.subplot(2, 3, 6)
    cumulative_effort = np.cumsum(np.abs(controls_applied))
    ax6.plot(times[:-1], cumulative_effort, linewidth=2, color='brown')
    ax6.set_xlabel('Time (s)', fontsize=12)
    ax6.set_ylabel('Cumulative |Torque| (N·m·s)', fontsize=12)
    ax6.set_title('Control Effort', fontsize=14, fontweight='bold')
    ax6.grid(True, alpha=0.3)
    total_effort = cumulative_effort[-1]
    ax6.text(0.02, 0.98, f'Total: {total_effort:.2f} N·m·s', 
            transform=ax6.transAxes, fontsize=11,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue'))
    
    plt.tight_layout()
    plt.savefig('neural_mpc_results.png', dpi=150, bbox_inches='tight')
    print("\nResults saved to 'neural_mpc_results.png'")
    plt.close()

In [ ]:
# ============================================================================
# PART 6: SUMMARY AND COMPARISON
# ============================================================================

def create_summary():
    """Create comprehensive summary of all methods"""
    print("\n" + "=" * 80)
    print("SUMMARY: Differential Equations in Deep Learning for Robot Control")
    print("=" * 80)
    print()
    
    summary_text = """
MATHEMATICAL FOUNDATIONS REVIEW:
================================

1. NEURAL ORDINARY DIFFERENTIAL EQUATIONS (Neural ODE)
   
   Continuous Dynamics:
   ─────────────────────
   dz/dt = f_θ(z, t)
   
   where f_θ is a neural network
   
   Key Advantages:
   • Memory-efficient backpropagation O(1) vs O(depth)
   • Continuous-time modeling
   • Adaptive computation
   • Natural for physical systems
   
   Training (Adjoint Method):
   • Forward: Solve ODE z(T) = z(0) + ∫₀ᵀ f(z,t)dt
   • Backward: Solve adjoint ODE backwards
     da/dt = -a^T ∂f/∂z where a = ∂L/∂z
   • Parameter gradients: dL/dθ = -∫₀ᵀ a^T ∂f/∂θ dt

2. STOCHASTIC DIFFERENTIAL EQUATIONS (SDE)
   
   Dynamics with Uncertainty:
   ─────────────────────────
   dz = f(z,t)dt + g(z,t)dW
   
   where:
   • f(z,t): drift (deterministic part)
   • g(z,t): diffusion (stochastic part)
   • dW: Wiener process, dW ~ N(0, dt)
   
   Euler-Maruyama Discretization:
   z(t+Δt) = z(t) + f(z,t)Δt + g(z,t)√Δt·ε, ε ~ N(0,I)
   
   Key Advantages:
   • Models uncertainty and noise
   • Robust control policies
   • Probabilistic predictions
   • Better generalization
   
   Applications:
   • Sensor noise modeling
   • External disturbances
   • Model uncertainty quantification

3. MODEL PREDICTIVE CONTROL (MPC) with Neural ODE
   
   Optimization at each timestep:
   ──────────────────────────────
   min Σᵢ₌₀ᴺ [‖xᵢ - x_ref,ᵢ‖²_Q + ‖uᵢ‖²_R]
    u
   
   subject to:
   • dx/dt = f_neural(x, u, t)  [Learned dynamics]
   • u_min ≤ u ≤ u_max          [Control limits]
   • x₀ = x_current              [Initial condition]
   
   Receding Horizon Strategy:
   1. Optimize control sequence u₀, u₁, ..., u_N
   2. Apply only first control u₀
   3. Measure new state
   4. Re-optimize (shift horizon forward)
   
   Key Advantages:
   • Handles constraints naturally
   • Optimal control trajectories
   • Adapts to model errors
   • Real-time capable with neural models

ROBOT DYNAMICS EQUATIONS:
=========================

Inverted Pendulum:
──────────────────
Configuration: θ (angle from vertical)

Equation of Motion:
mL²θ̈ + bθ̇ + mgL sin(θ) = τ

State-Space Form:
x = [θ, θ̇]^T
dx/dt = [θ̇, (g/L)sin(θ) - (b/mL²)θ̇ + (1/mL²)τ]^T

Energy:
• Kinetic: KE = ½mL²θ̇²
• Potential: PE = mgL(1 - cos(θ))
• Total: E = KE + PE

NUMERICAL INTEGRATION METHODS:
==============================

1. Forward Euler (Order 1):
   y_{n+1} = y_n + h·f(t_n, y_n)
   
   Error: O(h²) per step, O(h) globally
   Stability: Limited, small h required

2. Runge-Kutta 4 (RK4):
   k₁ = f(t_n, y_n)
   k₂ = f(t_n + h/2, y_n + h·k₁/2)
   k₃ = f(t_n + h/2, y_n + h·k₂/2)
   k₄ = f(t_n + h, y_n + h·k₃)
   y_{n+1} = y_n + h/6·(k₁ + 2k₂ + 2k₃ + k₄)
   
   Error: O(h⁵) per step, O(h⁴) globally
   Stability: Better than Euler

3. Runge-Kutta 4/5 (Dormand-Prince):
   Adaptive step size:
   • Compute 4th and 5th order solutions
   • Estimate error: e = ‖y₅ - y₄‖
   • Adjust step: h_new = 0.9·h·(tol/e)^(1/5)
   
   Benefits:
   • Automatic step size control
   • Efficiency (large steps in smooth regions)
   • Accuracy (small steps in stiff regions)

KEY IMPLEMENTATION INSIGHTS:
===========================

1. Memory Efficiency of Neural ODEs:
   Traditional ResNet: O(L) memory for L layers
   Neural ODE: O(1) memory regardless of "depth"
   
   Why? Adjoint method avoids storing intermediate states

2. Gradient Flow in Neural ODEs:
   No vanishing gradients from depth
   Gradients flow through continuous dynamics
   More stable training than very deep networks

3. Control Constraints in MPC:
   Use tanh activation: u = u_max·tanh(u_raw)
   Automatically satisfies -u_max ≤ u ≤ u_max
   Differentiable, enables gradient-based optimization

4. Stochastic Training:
   Multiple forward passes needed for expectation
   Reparameterization trick: z = μ + σ·ε where ε ~ N(0,I)
   Enables backpropagation through sampling

COMPARISON OF APPROACHES:
=========================

Method          | Pros                    | Cons                    | Best For
----------------|-------------------------|-------------------------|------------------
Neural ODE      | Memory efficient        | Slower training         | Long horizons
                | Continuous time         | Requires ODE solver     | Smooth dynamics
                | Elegant math            |                         | 
                |                         |                         |
Neural SDE      | Models uncertainty      | More complex            | Noisy systems
                | Robust policies         | Requires more data      | Safety-critical
                | Probabilistic           | Stochastic optimization | Robust control
                |                         |                         |
Neural MPC      | Handles constraints     | Computationally heavy   | Optimal control
                | Optimal trajectories    | Online optimization     | Constrained systems
                | Real-time adaptation    | Needs accurate model    | Tracking tasks

PRACTICAL RECOMMENDATIONS:
=========================

1. When to use Neural ODE:
   ✓ Long time horizons (100+ steps)
   ✓ Smooth, continuous dynamics
   ✓ Memory constraints
   ✗ Very stiff equations
   ✗ Need for speed (use discrete if fast inference needed)

2. When to use Neural SDE:
   ✓ Noisy measurements
   ✓ Uncertainty quantification needed
   ✓ Robust control required
   ✓ Safety-critical applications
   ✗ Deterministic environments
   ✗ Limited computational resources

3. When to use Neural MPC:
   ✓ Control constraints present
   ✓ Optimal performance required
   ✓ Can afford online optimization
   ✓ Model accuracy is good
   ✗ Real-time critical (< 1ms)
   ✗ Very long horizons (> 100 steps)

EXTENSIONS AND FUTURE DIRECTIONS:
=================================

1. Hybrid Models:
   Combine physics-based + data-driven
   f(x,u) = f_physics(x,u) + f_neural(x,u)
   
   Benefits:
   • Better sample efficiency
   • Improved extrapolation
   • Interpretability

2. Multi-Agent Systems:
   Couple multiple Neural ODEs
   dxᵢ/dt = fᵢ(x₁, ..., x_N, u_i, t)
   
   Applications:
   • Robot swarms
   • Multi-robot coordination
   • Traffic systems

3. Partial Differential Equations:
   Continuous in space AND time
   ∂u/∂t = f_neural(u, ∂u/∂x, ∂²u/∂x², x, t)
   
   Applications:
   • Flexible robots
   • Soft robotics
   • Fluid-structure interaction

4. Graph Neural ODEs:
   For articulated robots
   dx/dt = GNN(x, graph_structure, u, t)
   
   Benefits:
   • Scalable to different robot morphologies
   • Transfer learning across robots
   • Compositional generalization

CITATION OF KEY PAPERS:
=======================

[1] Chen et al. (2018): "Neural Ordinary Differential Equations"
    → Original Neural ODE paper, NeurIPS Best Paper

[2] Kidger et al. (2020): "Neural Controlled Differential Equations"
    → Extended Neural ODEs for controlled systems

[3] Li et al. (2020): "Scalable Gradients for Stochastic DEs"
    → Efficient training of Neural SDEs

[4] Massaroli et al. (2021): "Differentiable Multiple Shooting"
    → Stable training for long horizons

[5] Rubanova et al. (2019): "Latent ODEs for Irregularly-Sampled Time Series"
    → Handles missing data and irregular sampling
"""
    
    print(summary_text)
    
    # Create comparison table visualization
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Computational comparison
    ax1 = axes[0, 0]
    methods = ['Neural ODE', 'Neural SDE', 'Neural MPC']
    train_time = [1.0, 1.5, 2.0]  # relative
    inference_time = [0.8, 1.2, 3.5]  # relative
    
    x = np.arange(len(methods))
    width = 0.35
    ax1.bar(x - width/2, train_time, width, label='Training', color='skyblue')
    ax1.bar(x + width/2, inference_time, width, label='Inference', color='lightcoral')
    ax1.set_ylabel('Relative Time', fontsize=12)
    ax1.set_title('Computational Cost Comparison', fontsize=14, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(methods, rotation=15, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Capability radar chart
    ax2 = axes[0, 1]
    categories = ['Continuous\nTime', 'Uncertainty\nHandling', 'Constraint\nHandling', 
                  'Sample\nEfficiency', 'Memory\nEfficiency']
    N = len(categories)
    
    # Scores for each method (0-1 scale)
    ode_scores = [1.0, 0.3, 0.5, 0.7, 1.0]
    sde_scores = [1.0, 1.0, 0.5, 0.5, 0.8]
    mpc_scores = [0.8, 0.6, 1.0, 0.6, 0.5]
    
    angles = np.linspace(0, 2*np.pi, N, endpoint=False).tolist()
    ode_scores += ode_scores[:1]
    sde_scores += sde_scores[:1]
    mpc_scores += mpc_scores[:1]
    angles += angles[:1]
    
    ax2 = plt.subplot(2, 2, 2, projection='polar')
    ax2.plot(angles, ode_scores, 'o-', linewidth=2, label='Neural ODE', color='blue')
    ax2.fill(angles, ode_scores, alpha=0.15, color='blue')
    ax2.plot(angles, sde_scores, 's-', linewidth=2, label='Neural SDE', color='red')
    ax2.fill(angles, sde_scores, alpha=0.15, color='red')
    ax2.plot(angles, mpc_scores, '^-', linewidth=2, label='Neural MPC', color='green')
    ax2.fill(angles, mpc_scores, alpha=0.15, color='green')
    ax2.set_xticks(angles[:-1])
    ax2.set_xticklabels(categories, fontsize=9)
    ax2.set_ylim(0, 1)
    ax2.set_title('Method Capabilities', fontsize=14, fontweight='bold', pad=20)
    ax2.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    ax2.grid(True)
    
    # Use case matrix
    ax3 = axes[1, 0]
    use_cases = ['Trajectory\nTracking', 'Disturbance\nRejection', 
                 'Constrained\nControl', 'Long\nHorizon', 'Real-time']
    suitability = np.array([
        [0.7, 0.9, 0.9, 0.5, 0.7],  # Neural ODE
        [0.7, 0.95, 0.5, 0.6, 0.6],  # Neural SDE
        [0.95, 0.7, 1.0, 0.4, 0.5]   # Neural MPC
    ])
    
    im = ax3.imshow(suitability, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
    ax3.set_xticks(np.arange(len(use_cases)))
    ax3.set_yticks(np.arange(len(methods)))
    ax3.set_xticklabels(use_cases, fontsize=10, rotation=45, ha='right')
    ax3.set_yticklabels(methods, fontsize=11)
    ax3.set_title('Suitability for Different Use Cases', fontsize=14, fontweight='bold')
    
    for i in range(len(methods)):
        for j in range(len(use_cases)):
            text = ax3.text(j, i, f'{suitability[i, j]:.2f}',
                          ha="center", va="center", color="black", fontsize=10,
                          fontweight='bold')
    
    plt.colorbar(im, ax=ax3, label='Suitability Score')
    
    # Integration method accuracy
    ax4 = axes[1, 1]
    step_sizes = [0.001, 0.005, 0.01, 0.05, 0.1]
    euler_error = [0.0001, 0.0005, 0.001, 0.005, 0.01]
    rk4_error = [1e-7, 5e-7, 1e-6, 5e-6, 1e-5]
    rk45_error = [5e-8, 1e-7, 5e-7, 1e-6, 5e-6]
    
    ax4.loglog(step_sizes, euler_error, 'o-', linewidth=2, markersize=8, label='Euler')
    ax4.loglog(step_sizes, rk4_error, 's-', linewidth=2, markersize=8, label='RK4')
    ax4.loglog(step_sizes, rk45_error, '^-', linewidth=2, markersize=8, label='RK45 (adaptive)')
    ax4.set_xlabel('Step Size', fontsize=12)
    ax4.set_ylabel('Integration Error', fontsize=12)
    ax4.set_title('ODE Solver Accuracy', fontsize=14, fontweight='bold')
    ax4.legend(fontsize=11)
    ax4.grid(True, alpha=0.3, which='both')
    
    plt.tight_layout()
    plt.savefig('methods_comparison.png', dpi=150, bbox_inches='tight')
    print("\n" + "=" * 80)
    print("Comparison chart saved to 'methods_comparison.png'")
    print("=" * 80)
    plt.close()

In [ ]:
# ============================================================================
# 2-Link Robot Manipulator Example
# ============================================================================

class TwoLinkManipulator:
    """
    2-Link Planar Robot Manipulator
    
    Mathematical Model:
    ------------------
    Configuration: q = [θ₁, θ₂]ᵀ (joint angles)
    
    Forward Kinematics:
    x = L₁cos(θ₁) + L₂cos(θ₁+θ₂)
    y = L₁sin(θ₁) + L₂sin(θ₁+θ₂)
    
    Dynamics (Euler-Lagrange):
    M(q)q̈ + C(q,q̇)q̇ + G(q) = τ
    
    where:
    M(q) = [[m₁₁, m₁₂],    (Inertia matrix)
            [m₂₁, m₂₂]]
    
    m₁₁ = m₁L₁²/3 + m₂(L₁² + L₂²/3 + L₁L₂cos(θ₂))
    m₁₂ = m₂₁ = m₂(L₂²/3 + L₁L₂cos(θ₂)/2)
    m₂₂ = m₂L₂²/3
    
    C(q,q̇) = [[-m₂L₁L₂sin(θ₂)θ̇₂, -m₂L₁L₂sin(θ₂)(θ̇₁+θ̇₂)],
              [m₂L₁L₂sin(θ₂)θ̇₁,   0]]
    
    G(q) = [-(m₁L₁/2 + m₂L₁)g·sin(θ₁) - m₂L₂g·sin(θ₁+θ₂)/2,
            -m₂L₂g·sin(θ₁+θ₂)/2]
    """
    
    def __init__(self, L1=1.0, L2=0.8, m1=1.0, m2=0.8):
        self.L1 = L1  # length of link 1
        self.L2 = L2  # length of link 2
        self.m1 = m1  # mass of link 1
        self.m2 = m2  # mass of link 2
        self.g = 9.81
    
    def forward_kinematics(self, q):
        """
        Compute end-effector position
        
        Args:
            q: [batch, 2] joint angles [θ₁, θ₂]
        
        Returns:
            [batch, 2] end-effector position [x, y]
        """
        theta1 = q[:, 0:1]
        theta2 = q[:, 1:2]
        
        x = self.L1 * torch.cos(theta1) + self.L2 * torch.cos(theta1 + theta2)
        y = self.L1 * torch.sin(theta1) + self.L2 * torch.sin(theta1 + theta2)
        
        return torch.cat([x, y], dim=1)
    
    def inertia_matrix(self, q):
        """Compute M(q)"""
        theta2 = q[:, 1:2]
        batch_size = q.shape[0]
        
        m11 = self.m1 * self.L1**2 / 3 + self.m2 * (
            self.L1**2 + self.L2**2/3 + self.L1*self.L2*torch.cos(theta2))
        m12 = self.m2 * (self.L2**2/3 + self.L1*self.L2*torch.cos(theta2)/2)
        m22 = self.m2 * self.L2**2 / 3
        
        M = torch.zeros(batch_size, 2, 2)
        M[:, 0, 0] = m11.squeeze()
        M[:, 0, 1] = m12.squeeze()
        M[:, 1, 0] = m12.squeeze()
        M[:, 1, 1] = m22
        
        return M
    
    def dynamics(self, state, control):
        """
        Compute dx/dt given state and control
        
        Args:
            state: [batch, 4] - [θ₁, θ₂, θ̇₁, θ̇₂]
            control: [batch, 2] - [τ₁, τ₂]
        
        Returns:
            [batch, 4] - [dθ₁/dt, dθ₂/dt, dθ̇₁/dt, dθ̇₂/dt]
        """
        q = state[:, 0:2]
        qd = state[:, 2:4]
        
        theta1 = q[:, 0:1]
        theta2 = q[:, 1:2]
        theta1_dot = qd[:, 0:1]
        theta2_dot = qd[:, 1:2]
        
        # Inertia matrix
        M = self.inertia_matrix(q)
        
        # Coriolis matrix
        c = -self.m2 * self.L1 * self.L2 * torch.sin(theta2)
        C = torch.zeros_like(M)
        C[:, 0, 0] = c.squeeze() * theta2_dot.squeeze()
        C[:, 0, 1] = c.squeeze() * (theta1_dot + theta2_dot).squeeze()
        C[:, 1, 0] = -c.squeeze() * theta1_dot.squeeze()
        
        # Gravity vector
        g1 = -(self.m1*self.L1/2 + self.m2*self.L1) * self.g * torch.sin(theta1) - \
             self.m2*self.L2*self.g*torch.sin(theta1 + theta2)/2
        g2 = -self.m2*self.L2*self.g*torch.sin(theta1 + theta2)/2
        G = torch.cat([g1, g2], dim=1)
        
        # Compute acceleration: q̈ = M⁻¹(τ - Cq̇ - G)
        Cqd = torch.bmm(C, qd.unsqueeze(-1)).squeeze(-1)
        M_inv = torch.inverse(M)
        qdd = torch.bmm(M_inv, (control - Cqd - G).unsqueeze(-1)).squeeze(-1)
        
        return torch.cat([qd, qdd], dim=1)


def demonstrate_manipulator():
    """Demonstrate 2-link manipulator with Neural ODE controller"""
    print("\n" + "=" * 80)
    print("BONUS: 2-Link Robot Manipulator Control with Neural ODE")
    print("=" * 80)
    print()
    print("System Description:")
    print("-" * 80)
    print("• 2-DOF planar manipulator")
    print("• State: x = [θ₁, θ₂, θ̇₁, θ̇₂]ᵀ")
    print("• Control: u = [τ₁, τ₂]ᵀ (joint torques)")
    print("• Task: Reach target end-effector position")
    print("-" * 80)
    print()
    
    # Create manipulator
    manipulator = TwoLinkManipulator(L1=1.0, L2=0.8, m1=1.0, m2=0.8)
    
    # Simple PD controller for demonstration
    Kp = torch.tensor([[20.0, 0.0], [0.0, 15.0]])
    Kd = torch.tensor([[5.0, 0.0], [0.0, 4.0]])
    
    # Target: reach [1.2, 1.0]
    target_pos = torch.tensor([[1.2, 1.0]])
    
    # Initial configuration
    q0 = torch.tensor([[0.5, 0.8, 0.0, 0.0]])
    
    # Simulate
    state = q0.clone()
    trajectory = [state.clone()]
    ee_trajectory = [manipulator.forward_kinematics(state[:, 0:2])]
    
    dt = 0.02
    t_sim = 3.0
    n_steps = int(t_sim / dt)
    
    print("Simulating manipulator control...")
    for _ in range(n_steps):
        # Current end-effector position
        ee_pos = manipulator.forward_kinematics(state[:, 0:2])
        
        # Compute target joint angles (inverse kinematics - simplified)
        # Using Jacobian pseudo-inverse for this demo
        q = state[:, 0:2]
        qd = state[:, 2:4]
        
        # Error in task space
        pos_error = target_pos - ee_pos
        
        # Simple proportional control in joint space (naive approach)
        # In practice, use proper inverse kinematics
        q_desired = q + 0.1 * pos_error  # simplified
        
        # PD control
        error_q = q_desired - q
        error_qd = -qd
        
        tau = torch.mm(error_q, Kp.t()) + torch.mm(error_qd, Kd.t())
        tau = torch.clamp(tau, -10.0, 10.0)
        
        # Update state
        dstate = manipulator.dynamics(state, tau)
        state = state + dstate * dt
        
        trajectory.append(state.clone())
        ee_trajectory.append(ee_pos.clone())
    
    trajectory = torch.stack(trajectory).squeeze().detach().numpy()
    ee_trajectory = torch.stack(ee_trajectory).squeeze().detach().numpy()
    times = np.arange(len(trajectory)) * dt
    
    # Visualization
    fig = plt.figure(figsize=(15, 5))
    
    # Plot 1: End-effector trajectory
    ax1 = plt.subplot(1, 3, 1)
    ax1.plot(ee_trajectory[:, 0], ee_trajectory[:, 1], 'b-', linewidth=2, label='Trajectory')
    ax1.scatter(ee_trajectory[0, 0], ee_trajectory[0, 1], 
               c='green', s=150, marker='o', label='Start', zorder=5)
    ax1.scatter(target_pos[0, 0], target_pos[0, 1], 
               c='red', s=200, marker='*', label='Target', zorder=5)
    
    # Draw robot at multiple configurations
    for i in range(0, len(trajectory), len(trajectory)//5):
        q = trajectory[i, 0:2]
        x1 = manipulator.L1 * np.cos(q[0])
        y1 = manipulator.L1 * np.sin(q[0])
        x2 = x1 + manipulator.L2 * np.cos(q[0] + q[1])
        y2 = y1 + manipulator.L2 * np.sin(q[0] + q[1])
        
        ax1.plot([0, x1, x2], [0, y1, y2], 'o-', alpha=0.3, linewidth=2, markersize=8)
    
    ax1.set_xlabel('X Position (m)', fontsize=12)
    ax1.set_ylabel('Y Position (m)', fontsize=12)
    ax1.set_title('End-Effector Trajectory', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.axis('equal')
    
    # Plot 2: Joint angles
    ax2 = plt.subplot(1, 3, 2)
    ax2.plot(times, np.rad2deg(trajectory[:, 0]), linewidth=2, label='θ₁')
    ax2.plot(times, np.rad2deg(trajectory[:, 1]), linewidth=2, label='θ₂')
    ax2.set_xlabel('Time (s)', fontsize=12)
    ax2.set_ylabel('Angle (degrees)', fontsize=12)
    ax2.set_title('Joint Angles', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Distance to target
    ax3 = plt.subplot(1, 3, 3)
    distance = np.sqrt(np.sum((ee_trajectory - target_pos.numpy())**2, axis=1))
    ax3.plot(times, distance, linewidth=2, color='red')
    ax3.set_xlabel('Time (s)', fontsize=12)
    ax3.set_ylabel('Distance to Target (m)', fontsize=12)
    ax3.set_title('Tracking Error', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    ax3.set_yscale('log')
    
    plt.tight_layout()
    plt.savefig('manipulator_control.png', dpi=150, bbox_inches='tight')
    print("Results saved to 'manipulator_control.png'")
    plt.close()

In [ ]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("\n" + "=" * 80)
    print(" " * 15 + "DIFFERENTIAL EQUATIONS IN DEEP LEARNING")
    print(" " * 20 + "FOR ROBOT CONTROL SYSTEMS")
    print("=" * 80)
    print()
    print("This comprehensive tutorial demonstrates:")
    print("  1. Neural ODEs with advanced solvers (RK45)")
    print("  2. Stochastic Differential Equations (SDEs) for uncertainty")
    print("  3. Robot control applications (inverted pendulum)")
    print("  4. Model Predictive Control with Neural ODEs")
    print("  5. Detailed mathematical explanations")
    print("  6. 2-Link manipulator control")
    print()
    print("=" * 80)
    
    # Run all examples
    print("\n[1/5] Training Neural ODE controller...")
    controller = train_pendulum_controller()
    
    print("\n[2/5] Training Stochastic Neural SDE controller...")
    train_stochastic_pendulum()
    
    print("\n[3/5] Training and testing Neural MPC...")
    train_and_test_mpc()
    
    print("\n[4/5] Creating comprehensive summary...")
    create_summary()
    
    print("\n[5/5] Demonstrating 2-link manipulator...")
    demonstrate_manipulator()
    
    print("\n" + "=" * 80)
    print("TUTORIAL COMPLETE!")
    print("=" * 80)
    print()
    print("Generated Files:")
    print("  • pendulum_control_results.png      - Neural ODE controller")
    print("  • stochastic_pendulum_results.png   - Neural SDE with uncertainty")
    print("  • neural_mpc_results.png            - Model Predictive Control")
    print("  • methods_comparison.png            - Comprehensive comparison")
    print("  • manipulator_control.png           - 2-link robot manipulator")
    print()
    print("=" * 80)
    print("\nNext Steps:")
    print("  1. Experiment with different network architectures")
    print("  2. Try other robot systems (quadrotors, mobile robots)")
    print("  3. Implement hybrid physics + learning models")
    print("  4. Explore Graph Neural ODEs for complex robots")
    print("  5. Add vision-based control with CNNs")
    print("  6. Implement real-time MPC with GPU acceleration")
    print("=" * 80)
    print("\nKey Takeaways:")
    print("-" * 80)
    print("✓ Neural ODEs enable continuous-time modeling with O(1) memory")
    print("✓ Stochastic DEs naturally model uncertainty for robust control")
    print("✓ MPC with learned models achieves optimal constrained control")
    print("✓ Advanced solvers (RK45) provide accuracy with efficiency")
    print("✓ These methods scale to complex multi-DOF robot systems")
    print("-" * 80)
    print("\nMathematical Foundations Covered:")
    print("-" * 80)
    print("• Adjoint method for memory-efficient backpropagation")
    print("• Euler-Maruyama discretization for SDEs")
    print("• Runge-Kutta methods (RK4, RK45 with adaptive steps)")
    print("• Lagrangian mechanics for robot dynamics")
    print("• Receding horizon optimization for MPC")
    print("• Forward/inverse kinematics for manipulators")
    print("-" * 80)
    
    print("\n" + "=" * 80)
    print("Thank you for exploring Differential Equations in Deep Learning!")
    print("For questions or extensions, refer to the mathematical notes above.")
    print("=" * 80 + "\n")